In [2]:
from tensorflow.keras import models, optimizers, losses
from utils import *
from loss import *
from network import *
from metrics import *
from ipywidgets import interact
from matplotlib import pyplot as plt
import cv2
import pandas as pd

In [3]:
def plot_result(y_pred, y_true, mode=12):
    # img : [Slices, Height, Width, 12]
    b, h, w, _ = y_pred.shape
    
    if mode == 12:
        tmp_pred = y_pred[...,3:-3]
        tmp_pred = np.transpose(tmp_pred, [0, 3, 1, 2])
        tmp_pred = np.reshape(tmp_pred, [b*6, h, w])

        tmp_true = y_true[...,3:-3]
        tmp_true = np.transpose(tmp_true, [0, 3, 1, 2])
        tmp_true = np.reshape(tmp_true, [b*6, h, w])
    
    elif mode ==6:
        tmp_pred = np.transpose(y_pred, [0, 3, 1, 2])
        tmp_pred = np.reshape(tmp_pred, [b*6, h, w])
        
        tmp_true = np.transpose(y_true, [0, 3, 1, 2])
        tmp_true = np.reshape(tmp_true, [b*6, h, w])
        
        
    max_idx = len(tmp_pred)-1
    err = tmp_true-tmp_pred
    err_min = (tmp_true-tmp_pred).min()
    err_max = (tmp_true-tmp_pred).max()
    
    def plot(idx=0):
        plt.figure(figsize=(12, 6))
        plt.subplot(131)
        plt.title("y_pred")
        plt.imshow(tmp_pred[idx], cmap='gray', vmin=0, vmax=tmp_true.max())
        plt.xlabel("%.2f"%(tmp_pred[idx].max()))
        plt.subplot(132)
        plt.title("y_true")
        plt.imshow(tmp_true[idx], cmap='gray', vmax=tmp_true.max())
        plt.xlabel("%.2f"%(tmp_true[idx].max()))
        plt.subplot(133)
        plt.title("Error (true - pred)")
        plt.imshow(err[idx], cmap='gray', vmin = err_min, vmax=err_max)
        #plt.xlabel("%.2f ~ %.2f"%(err[idx].min(), err[idx].max()))
    interact(plot, idx=(0, max_idx, 1))

In [5]:
train_low, train_high, val_low, val_high, test_low = data_loader_v2('../interpolation/')
print("Train X's shape : ", train_low.shape)
print("Train Y's shape : ", train_high.shape)
print("Validation X's shape : ", val_low.shape)
print("Validation Y's shape : ", val_high.shape)
print("Test X's shape : ", test_low.shape)


Train X's shape :  (303, 320, 256, 3)
Train Y's shape :  (303, 320, 256, 6)
Validation X's shape :  (25, 320, 256, 3)
Validation Y's shape :  (25, 320, 256, 6)
Test X's shape :  (328, 320, 256, 3)


In [6]:
root_path = './checkpoint/3to6/unet_l2_mi/'

model_path = root_path+'model.json'
train_csv_path = root_path+'train_loss.csv'
val_csv_path = root_path+'val_loss.csv'

weights_list = sorted(os.listdir(root_path))
weights_list = [i for i in weights_list if '.h5' in i]

In [7]:
weights_list

['0100_82120.15_34930.64.h5',
 '0200_39371.41_36263.26.h5',
 '0300_24488.82_36686.42.h5',
 '0400_18782.08_34189.55.h5',
 '0500_13827.60_35558.38.h5',
 '0600_12550.16_34438.00.h5',
 '0700_10482.90_34399.44.h5',
 '0800_10921.56_33133.08.h5',
 '0900_10387.04_34459.57.h5',
 '1000_8680.17_34307.16.h5',
 '1100_9925.18_34100.65.h5',
 '1200_7942.00_32596.69.h5',
 '1300_7960.06_35849.45.h5',
 '1400_6229.91_33849.95.h5',
 '1500_5913.94_33407.04.h5']

In [8]:
with open(model_path, 'r') as f:
    model_json = f.read()

A = models.model_from_json(model_json)

A.load_weights(root_path+weights_list[-1])

# A = models.load_model(root_path+'model.h5', compile=False)
# A.load_weights(root_path+weights_list[-2])

# A = models.load_model(root_path+weights_list[-1], compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
train_, _ = A.predict(train_low)
val_, _ = A.predict(val_low)
# train_ = A.predict(train_low)
# val_ = A.predict(val_low)
print(train_.shape)
print(val_.shape)

(303, 320, 256, 6)
(25, 320, 256, 6)


In [21]:
real_vmax = train_high.max(axis=(1, 2), keepdims=True)
eval_vmax = train_.max(axis=(1, 2), keepdims=True)
print(real_vmax.shape)
print(eval_vmax.shape)

(303, 1, 1, 6)
(303, 1, 1, 6)


In [28]:
test_vmax = np.maximum(real_vmax, eval_vmax)

ValueError: invalid number of arguments

In [29]:
print(real_vmax[0].ravel())
print(eval_vmax[0].ravel())
print(test_vmax[0].ravel())

[1368.81632585 1329.85762123 1348.66527174 1252.94776469 1267.61325408
 1382.13841163]
[1364.691  1329.3895 1353.215  1279.6041 1277.6371 1339.1931]
[1368.81632585 1329.85762123 1353.21496582 1279.60412598 1277.63708496
 1382.13841163]


In [10]:
vmax = train_high.max(axis=(1, 2), keepdims=True)
train_mae = MAE(train_high/vmax*255, train_/vmax*255)
train_mse = MSE(train_high/vmax*255, train_/vmax*255)
train_rmse = RMSE(train_high/vmax*255, train_/vmax*255)
train_psnr = PSNR(train_high, train_)
train_ssim = SSIM(train_high, train_)
#train_mi = MI(train_high, train_)
#train_mi = np.nan_to_num(train_mi)
#train_nmi = NMI(train_high, train_)
#train_nmi = np.nan_to_num(train_nmi)

print('MAE : ', train_mae.shape, train_mae.min(), train_mae.mean(), train_mae.max(), train_mae.std())
print('MSE : ', train_mse.shape, train_mse.min(), train_mse.mean(), train_mse.max(), train_mse.std())
print('RMSE : ', train_rmse.shape, train_rmse.min()/255*100, train_rmse.mean()/255*100, train_rmse.max()/255*100, train_rmse.std())
print('PSNR : ', train_psnr.shape, train_psnr.min(), train_psnr.mean(), train_psnr.max(), train_psnr.std())
print('SSIM : ', train_ssim.shape, train_ssim.min(), train_ssim.mean(), train_ssim.max(), train_ssim.std())
#print(train_ssim.shape, train_ssim.mean(), train_ssim.std())
#print(train_mi.shape, train_mi.mean(), train_mi.std())
#print(train_nmi.shape, train_nmi.mean(), train_nmi.std())

MAE :  (303, 6) 0.33270122162957694 0.9235502523202491 1.8162484558863856 0.2814309878816606
MSE :  (303, 6) 0.7251329132715362 3.001669037523104 9.52627440493016 1.5872996104786665
RMSE :  (303, 6) 0.8515473640799648 1.6763226759512553 3.0864663297904547 0.4377343071027518
PSNR :  (303, 6) 33.35829946781035 42.75334958386953 48.838610648891034 2.4381374005806804


In [13]:
print('MAE : ', train_mae.shape, train_mae.min(), train_mae.mean(), train_mae.max(), train_mae.std())
print('MSE : ', train_mse.shape, train_mse.min(), train_mse.mean(), train_mse.max(), train_mse.std())
print('RMSE : ', train_rmse.shape, train_rmse.min()/255*100, train_rmse.mean()/255*100, train_rmse.max()/255*100, train_rmse.std())
print('PSNR : ', train_psnr.shape, train_psnr.min(), train_psnr.mean(), train_psnr.max(), train_psnr.std())
print('SSIM : ', train_ssim.shape, train_ssim.min(), train_ssim.mean(), train_ssim.max(), train_ssim.std())

MAE :  (303, 6) 0.33270122162957694 0.9235502523202491 1.8162484558863856 0.2814309878816606
MSE :  (303, 6) 0.7251329132715362 3.001669037523104 9.52627440493016 1.5872996104786665
RMSE :  (303, 6) 0.3339401427764568 0.657381441549512 1.2103789528590017 0.4377343071027518
PSNR :  (303, 6) 33.35829946781035 42.75334958386953 48.838610648891034 2.4381374005806804
SSIM :  (303, 6) 0.9787654737559222 0.9913499609484603 0.9968139144466056 0.002880365796372305


In [14]:
vmax = val_high.max(axis=(1, 2), keepdims=True)
val_mae = MAE(val_high/vmax*255, val_/vmax*255)
val_mse = MSE(val_high/vmax*255, val_/vmax*255)
val_rmse = RMSE(val_high/vmax*255, val_/vmax*255)
val_psnr = PSNR(val_high, val_)
val_ssim = SSIM(val_high, val_)
# val_mi = MI(val_high, val_)
# val_mi = np.nan_to_num(val_mi)
# val_nmi = NMI(val_high, val_)
# val_nmi = np.nan_to_num(val_nmi)

print('MAE : ', val_mae.shape, val_mae.min(), val_mae.mean(), val_mae.max(), val_mae.std())
print('MSE : ', val_mse.shape, val_mse.min(), val_mse.mean(), val_mse.max(), val_mse.std())
print('RMSE : ', val_rmse.shape, val_rmse.min(), val_rmse.mean(), val_rmse.max(), val_rmse.std())
print('PSNR : ', val_psnr.shape, val_psnr.min(), val_psnr.mean(), val_psnr.max(), val_psnr.std())
print('SSIM : ', val_ssim.shape, val_ssim.min(), val_ssim.mean(), val_ssim.max(), val_ssim.std())
# print(val_mi.shape, val_mi.mean(), val_mi.std())
# print(val_nmi.shape, val_nmi.mean(), val_nmi.std())

MAE :  (25, 6) 1.9136277287621788 3.945708865779985 7.953327421332813 1.4670159768363369
MSE :  (25, 6) 17.804707038532506 80.73966797968147 266.25263629192494 53.644362085251935
RMSE :  (25, 6) 4.21956242263727 8.543982124668617 16.31724965464232 2.782092276151999
PSNR :  (25, 6) 29.37351063802952 33.8186065066202 37.364726247534456 1.8634304940416189
SSIM :  (25, 6) 0.8183164341772178 0.9102117634697442 0.9690585335360947 0.03769176421932355


$$
MSE = \frac{1}{n} \sum^n{(y-\hat{y})^2}
$$

$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum^n{(y-\hat{y})^2}}
$$

$$
RMSPE = \sqrt{\frac{1}{n} \sum^n({\frac{y-\hat{y}}{y}})^2}
$$

$$
PSNR = 10\log_{10}(\frac{MAX}{\sqrt{MSE}}) = 20\log_{10}{MAX} - 10\log_{10}\sqrt{MSE}
$$

$$
SSIM = I(x,y)C(x,y)S(x,y)
$$

In [ ]:
np.save(root_path+'train_result', train_)
np.save(root_path+'val_result', val_)

In [ ]:
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import normalized_mutual_info_score as nmi

In [ ]:
def mutual_information_np(y_true, y_pred):
    hist2d, _, _ = np.histogram2d(y_true.ravel(), y_pred.ravel(), bins=255, range=[[0,255],[0,255]])    
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    px_py = px[:, None] * py[None, :] # Broadcast to multiply marginals    
    nzs = pxy > 0 # Only non-zero pxy values contribute to the sum
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

In [ ]:
def mutual_information_new(y_true, y_pred):
    hist2d, _, _ = np.histogram2d(y_true.ravel(), y_pred.ravel(), bins=255, range=[[0,255],[0,255]])    
    pxy = hist2d / float(np.sum(hist2d))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    hx = -np.sum(px*np.log(px))
    hy = -np.sum(py*np.log(py))
    hxy = -np.sum(pxy*np.log(pxy))
    px_py = px[:, None] * py[None, :]
    print(px.shape, py.shape, px_py.shape)
    return 2*(hx+hy-hxy)/(hx+hy)

In [ ]:
mutual_information_new(tmp_true, tmp_pred)

In [ ]:
b = 0
sli = 0
tmp_true = train_high[b, ..., sli]/train_high[b, ..., sli].max()*255
tmp_pred = train_[b, ..., sli]/train_high[b, ..., sli].max()*255
print(mi(tmp_true.ravel(), tmp_true.ravel()), mutual_information_np(tmp_true, tmp_true))
print(nmi(tmp_true.ravel(), tmp_true.ravel()))
print(mutual_information_new(tmp_true, tmp_pred))

In [ ]:
test10 = val_high/val_high.max(axis=(1, 2), keepdims=True)
test11 = val_/val_.max(axis=(1, 2), keepdims=True)
test00 = train_high/train_high.max(axis=(1, 2), keepdims=True)
test01 = train_/train_.max(axis=(1, 2), keepdims=True)

In [ ]:
plot_result(train_, train_high, 6)

In [ ]:
plot_result(val_, val_high, 6)

In [ ]:
train_loss = pd.read_csv(train_csv_path)
val_loss = pd.read_csv(val_csv_path)

In [ ]:
plt.plot(train_loss.Generator_Style[:100])
plt.plot(val_loss.Generator_Style[:100])
plt.ylim((0, 10000))